In [64]:
IMAGE_SIZE = [224, 224]
TRAINING_PATH = '/home/dilara/UGI-endoscopy/training-data'
TESTING_PATH = '/home/dilara/UGI-endoscopy/testing-data'
CLASSES = ['other2', 'pylorus', 'z-line', 'retroflex-stomach']

import keras
from keras.utils import np_utils
from keras.layers import Dense, Flatten 
from keras.models import Model, Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import glob
import matplotlib.pyplot as plt
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import skimage
from skimage import io
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sbs
from PIL import Image

def get_paths(path):
## setting the paths
    all_paths = []
    for landmark in CLASSES:
        other_path = os.path.join(path, landmark, '*')
        all_paths.append(sorted(glob.glob(other_path)))
    return all_paths
# all_paths = list(np.concatenate(all_paths).flat)

def get_class_num(path):
## finding the number of image per class
    len_lis = []
    for lis in path:
        x = 0
        for item in lis:
            x+=1
        len_lis.append(x)
    print(len_lis)
    return len_lis

def create_labels(len_lis):
    labels_lis = []
    for i in range(4):
        labels_lis.append([i] * len_lis[i])
        all_labels = list(np.concatenate(labels_lis).flat)
        all_labels = np.array(all_labels)
    return all_labels

all_paths_train = get_paths(TRAINING_PATH)
all_paths_test = get_paths(TESTING_PATH)
len_lis_train = get_class_num(all_paths_test)
len_lis_test = get_class_num(all_paths_train)

all_paths_train = list(np.concatenate(all_paths_train).flat)
all_paths_test = list(np.concatenate(all_paths_test).flat)

all_labels_train = create_labels(len_lis_train)
all_labels_test = create_labels(len_lis_test)

[3260, 586, 147, 230]
[6321, 6993, 6524, 5348]


In [16]:
## Augmenting the Dataset
# for path in all_paths:
# 	file_name = path[:len(path)-4]
# 	image = cv2.imread(path)
# 	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# 	rows, cols, dim = image.shape

# 	# flipping images upside down and leftright
# 	image_fliplr = np.fliplr(image)
# 	image_flipud = np.flipud(image)
# 	plt.imsave(f'{file_name}_image_flipud.jpg', image_flipud)
# 	plt.imsave(f'{file_name}_image_fliplr.jpg', image_fliplr)

# 	# rotating image
# 	rotated_img = skimage.transform.rotate(image, 90)
# 	plt.imsave(f'{file_name}_rotated_img.jpg', rotated_img)

# 	# shear transform on x-axis
# 	M = np.float32([[1, 0.7, 0],
# 					[0, 1  , 0],
# 					[0, 0  , 1]])             
# 	sheared_img = cv2.warpPerspective(image,M,(int(cols*1.5),int(rows*1.5)))
# 	plt.imsave(f'{file_name}_sheared_img.jpg', sheared_img)

# 	## zoom out
# 	M = np.float32([[1.5, 0  , 0],
# 					[0,   1.5, 0],
# 					[0,   0,   1]])
# 	scaledin_img = cv2.warpPerspective(image,M,(cols,rows))
# 	plt.imsave(f'{file_name}_scaledin_img.jpg', scaledin_img)

# 	M = np.float32([[0.8, 0  , 0],
# 					[0,   0.8, 0],
# 					[0,   0,   1]])
# 	scaledout_img = cv2.warpPerspective(image,M,(cols,rows))
# 	plt.imsave(f'{file_name}_scaledout_img.jpg', scaledout_img)

In [38]:
# # Renaming the images
# all_paths_test = []

# for landmark in CLASSES:
#     other_path = os.path.join(TESTING_PATH, landmark, '*')
#     all_paths_test.append(sorted(glob.glob(other_path)))

# all_paths_test = list(np.concatenate(all_paths_test).flat)

# for path in all_paths_test:
#     old = path
#     new = f'{path[:len(path)-4]}_video5.jpg'
#     os.rename(old, new)

In [65]:
pix = []

def get_pix(path_lis):
    for path in path_lis:
        file_name = path.split('/')[-1]
        image = cv2.imread(path)
        image_data = cv2.resize(image, IMAGE_SIZE, interpolation = cv2.INTER_AREA)
        pix.append(image_data)
    return pix

pix_train = get_pix(all_labels_train)
pix_test = get_pix(all_labels_test)

label_train = keras.utils.np_utils.to_categorical(all_labels_train, num_classes=4)
label_test = keras.utils.np_utils.to_categorical(all_labels_test, num_classes=4)

    # pix = np.array(pix)
    # print(len(pix))
    # pix_train, pix_val, label_train, label_val = train_test_split(pix, all_labels, train_size=0.75, random_state = 42)
    # label_train = keras.utils.np_utils.to_categorical(label_train, num_classes=4)
    # label_val = keras.utils.np_utils.to_categorical(label_val, num_classes=4)
    # print(pix_train.shape)

AttributeError: 'numpy.int64' object has no attribute 'split'